In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import anndata as ad
import pandas as pd
import scanpy as sc
import numpy as np

import seaborn as sns; sns.set(color_codes=True)

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [26]:
import os
os.chdir('/Users/melnike/Desktop/StarDict')

In [27]:
# Read annotation measurements
annotDF = pd.read_csv('CRC_annotation_measurements.tsv', sep='\t')
annotDF.reset_index(drop=True,inplace=True)
annotDF.head()

,Image,Object ID,Name,Class,Parent,ROI,Centroid X µm,Centroid Y µm,Num Detections,Area µm^2,Perimeter µm
0,CRC.ome.tif,e837112d-e9e0-489d-9c3e-aec9db387e5f,PathAnnotationObject,NaN,Image,Geometry,741.3,698,18256,1290700.3,4108.3


In [28]:
# Read detection measurements
detectDF = pd.read_csv('CRC_measurements.tsv', sep='\t')
detectDF.reset_index(drop=True,inplace=True)
detectDF.head()

,Image,Object ID,Name,Class,Parent,ROI,Centroid X µm,Centroid Y µm,Nucleus: Area µm^2,Nucleus: Length µm,...,CD45 - TRITC: Membrane: Mean,CD45 - TRITC: Membrane: Median,CD45 - TRITC: Membrane: Min,CD45 - TRITC: Membrane: Max,CD45 - TRITC: Membrane: Std.Dev.,CD45 - TRITC: Cell: Mean,CD45 - TRITC: Cell: Median,CD45 - TRITC: Cell: Min,CD45 - TRITC: Cell: Max,CD45 - TRITC: Cell: Std.Dev.
0,CRC.ome.tif,e9d527e4-b86f-4ad6-9a7f-d5506999661a,PathCellObject,NaN,PathAnnotationObject,Polygon,655.53,96.46,27.1063,20.4408,...,522.2727,403.0,137,978,288.8788,486.3333,431.0,31,1029,245.4246
1,CRC.ome.tif,5578f442-6aa4-491c-a5f6-49822a8b4905,PathCellObject,NaN,PathAnnotationObject,Polygon,663.42,96.77,39.2610,23.0666,...,865.9344,847.0,199,2093,456.5124,856.7125,878.5,118,2093,367.9206
2,CRC.ome.tif,61e47480-6dc9-4030-bf41-e64c5baa97b9,PathCellObject,NaN,PathAnnotationObject,Polygon,623.61,101.06,36.5121,22.0329,...,718.8983,465.0,251,2142,502.3028,870.8338,746.0,210,2545,530.3132
3,CRC.ome.tif,a7d6f536-18cb-4fa4-9f87-8166e9c847e8,PathCellObject,NaN,PathAnnotationObject,Polygon,638.75,103.92,15.6478,14.9866,...,657.8958,466.5,35,1524,478.0514,600.2435,544.5,12,1524,404.0042
4,CRC.ome.tif,305e9c24-a34a-4ed0-9403-b7d5cef6a0b6,PathCellObject,NaN,PathAnnotationObject,Polygon,661.34,104.81,9.8856,11.5496,...,1366.5135,1386.0,650,2307,434.7817,1274.6077,1223.0,650,2307,321.2902


In [29]:
# Display total number of cells across the two datasets
detectDF.shape

(18256, 441)

In [30]:
# Extract the columns with the mean protein expression values
# This includes DAPI and Autofluorescence, which we will remove later
protDF = detectDF[[col for col in detectDF.columns if 'Mean' in col and 'Cell:' in col]]
protDF.head()

,DAPI: Cell: Mean,FoxP3 - Cy5: Cell: Mean,CD68 - TRITC: Cell: Mean,aSMA - Cy5: Cell: Mean,CD31 - TRITC: Cell: Mean,CD38 - Cy5: Cell: Mean,IDO-1 - TRITC: Cell: Mean,s100 - Cy5: Cell: Mean,CD11c - TRITC: Cell: Mean,PD-L1 - Cy5: Cell: Mean,...,CD8 - Cy5: Cell: Mean,PD-1 - TRITC: Cell: Mean,CD4 - Cy5: Cell: Mean,PanCK - TRITC: Cell: Mean,CD3 - Cy5: Cell: Mean,CD20 - TRITC: Cell: Mean,CD16 - Cy5: Cell: Mean,HLA-DR - TRITC: Cell: Mean,Vimentin - Cy5: Cell: Mean,CD45 - TRITC: Cell: Mean
0,3119.7317,280.5402,3356.2415,305.3333,313.7088,237.1456,5236.8125,535.7165,542.4636,1012.5364,...,535.6322,545.0996,592.6016,371.1762,450.0575,218.3103,1277.3027,1118.2605,2941.6858,486.3333
1,4214.8032,402.7250,2727.8813,479.1031,382.6656,283.9438,6774.6313,750.7156,1237.4594,1570.3594,...,925.1406,715.0000,811.3812,585.1281,823.2969,368.1187,2334.7812,1651.7875,4009.3782,856.7125
2,9569.4854,345.8556,2044.5286,445.9455,378.2697,318.9292,778.6785,582.3787,283.8774,1386.7030,...,1499.3896,639.4959,794.0027,1251.2698,1760.3951,256.0000,1443.3188,1187.5259,3746.3706,870.8338
3,2873.2566,363.1478,14340.7959,444.6174,439.9391,309.0261,837.1044,621.6087,274.4000,1103.4348,...,683.5913,555.3304,1127.2217,366.3000,564.9783,268.0522,2245.0347,1920.3914,5689.4478,600.2435
4,4968.7617,537.2462,5440.1309,930.9231,501.5769,374.4462,1315.2924,882.2692,2084.3691,2520.0000,...,924.1923,619.8231,1271.0538,1057.6154,717.5923,316.6692,3147.0923,3644.1384,6724.7998,1274.6077


In [31]:
# clean-up column names for easy access
newNames = {col:col.split(':')[0].split(' ')[0] for col in protDF.columns}
protDF.rename(newNames, inplace=True, axis=1)
protDF.head()

,DAPI,FoxP3,CD68,aSMA,CD31,CD38,IDO-1,s100,CD11c,PD-L1,...,CD8,PD-1,CD4,PanCK,CD3,CD20,CD16,HLA-DR,Vimentin,CD45
0,3119.7317,280.5402,3356.2415,305.3333,313.7088,237.1456,5236.8125,535.7165,542.4636,1012.5364,...,535.6322,545.0996,592.6016,371.1762,450.0575,218.3103,1277.3027,1118.2605,2941.6858,486.3333
1,4214.8032,402.7250,2727.8813,479.1031,382.6656,283.9438,6774.6313,750.7156,1237.4594,1570.3594,...,925.1406,715.0000,811.3812,585.1281,823.2969,368.1187,2334.7812,1651.7875,4009.3782,856.7125
2,9569.4854,345.8556,2044.5286,445.9455,378.2697,318.9292,778.6785,582.3787,283.8774,1386.7030,...,1499.3896,639.4959,794.0027,1251.2698,1760.3951,256.0000,1443.3188,1187.5259,3746.3706,870.8338
3,2873.2566,363.1478,14340.7959,444.6174,439.9391,309.0261,837.1044,621.6087,274.4000,1103.4348,...,683.5913,555.3304,1127.2217,366.3000,564.9783,268.0522,2245.0347,1920.3914,5689.4478,600.2435
4,4968.7617,537.2462,5440.1309,930.9231,501.5769,374.4462,1315.2924,882.2692,2084.3691,2520.0000,...,924.1923,619.8231,1271.0538,1057.6154,717.5923,316.6692,3147.0923,3644.1384,6724.7998,1274.6077


In [32]:
# Remove DAPI and Autofluorescence
protDF = protDF.drop(['DAPI'], axis=1)
protDF

,FoxP3,CD68,aSMA,CD31,CD38,IDO-1,s100,CD11c,PD-L1,Ki67,CD8,PD-1,CD4,PanCK,CD3,CD20,CD16,HLA-DR,Vimentin,CD45
0,280.5402,3356.2415,305.3333,313.7088,237.1456,5236.8125,535.7165,542.4636,1012.5364,250.8506,535.6322,545.0996,592.6016,371.1762,450.0575,218.3103,1277.3027,1118.2605,2941.6858,486.3333
1,402.7250,2727.8813,479.1031,382.6656,283.9438,6774.6313,750.7156,1237.4594,1570.3594,447.4688,925.1406,715.0000,811.3812,585.1281,823.2969,368.1187,2334.7812,1651.7875,4009.3782,856.7125
2,345.8556,2044.5286,445.9455,378.2697,318.9292,778.6785,582.3787,283.8774,1386.7030,469.9210,1499.3896,639.4959,794.0027,1251.2698,1760.3951,256.0000,1443.3188,1187.5259,3746.3706,870.8338
3,363.1478,14340.7959,444.6174,439.9391,309.0261,837.1044,621.6087,274.4000,1103.4348,294.2913,683.5913,555.3304,1127.2217,366.3000,564.9783,268.0522,2245.0347,1920.3914,5689.4478,600.2435
4,537.2462,5440.1309,930.9231,501.5769,374.4462,1315.2924,882.2692,2084.3691,2520.0000,391.6000,924.1923,619.8231,1271.0538,1057.6154,717.5923,316.6692,3147.0923,3644.1384,6724.7998,1274.6077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18251,359.5555,1695.9573,480.3932,278.3675,311.3248,887.1368,541.6496,245.8120,910.8205,364.3333,776.1539,733.3761,709.8547,10706.2305,644.0513,525.7436,1402.4786,12201.4961,2204.2563,798.8290
18252,310.5658,1042.1052,337.0132,218.9868,245.6447,704.7763,428.8026,149.8026,689.2500,249.7500,556.2632,610.3947,591.2632,5985.8555,448.8684,287.9868,1085.2368,3958.3816,2271.0527,490.4605
18253,397.0089,1470.1823,535.3822,272.8311,310.0533,1136.7289,594.8400,274.5555,1134.5688,399.5778,707.9911,710.8845,709.3911,10425.7910,746.8578,586.8267,1069.2844,11665.8535,1753.6666,1353.2622
18254,315.6187,1441.0509,487.8136,467.3305,241.2797,896.2966,477.0085,269.3305,1468.2457,283.6356,660.6949,782.1102,920.6780,10714.0000,887.6949,509.0508,1505.0593,5970.8896,1341.8813,759.9153


In [33]:
print(type(protDF))

<class 'pandas.core.frame.DataFrame'>


In [34]:
for column in protDF.columns:
    protDF[column] = protDF[column].sort_values(ascending=False).values

# Print the DataFrame with columns sorted in ascending order
print(protDF)

            FoxP3        CD68        aSMA        CD31        CD38       IDO-1  \
0      20819.4883  47270.9648  15869.7441  23830.9355  11181.6133  30254.2695   
1      19778.8750  37767.7109  15090.3350  20353.6406  10203.3330  23875.6797   
2      18885.8496  37407.5742  14762.7910  19358.8652   9865.8662  23537.9043   
3      18783.9746  37321.4570  13727.9502  18527.7422   8804.4014  22524.9434   
4      18246.5215  35034.5547  12635.0176  16731.4727   8706.7080  21567.5586   
...           ...         ...         ...         ...         ...         ...   
18251    225.6293    484.6548    180.6735     89.8564    125.3354    263.1338   
18252    224.0124    469.6838    178.5054     89.6250    111.5333    261.0949   
18253    211.5047    424.0095    167.6415     71.3487    110.6162    255.2355   
18254    206.2522    415.6729    162.5913     67.9809     97.8596    235.3679   
18255    191.2767    411.7123     64.1749     64.5354     91.4826    146.1130   

             s100      CD11

In [35]:
spatialCols = ['Centroid X µm','Centroid Y µm','Cell: Area µm^2','Image','Object ID']
spatialNames  = {'Centroid X µm':'spatial_X','Centroid Y µm': 'spatial_Y','Cell: Area µm^2': 'Area','Image':'ImageID','Object ID':'Object ID'}
spatDF = detectDF[spatialCols]
spatDF.rename(columns=spatialNames, inplace=True)
# clean up ImageID column by removing the ome.tif file extension
spatDF['ImageID'] = spatDF['ImageID'].apply(lambda x: x.split('.')[0])
spatDF

,spatial_X,spatial_Y,Area,ImageID,Object ID
0,655.53,96.46,64.8720,CRC,e9d527e4-b86f-4ad6-9a7f-d5506999661a
1,663.42,96.77,81.4933,CRC,5578f442-6aa4-491c-a5f6-49822a8b4905
2,623.61,101.06,92.2488,CRC,61e47480-6dc9-4030-bf41-e64c5baa97b9
3,638.75,103.92,57.1781,CRC,a7d6f536-18cb-4fa4-9f87-8166e9c847e8
4,661.34,104.81,33.1126,CRC,305e9c24-a34a-4ed0-9403-b7d5cef6a0b6
...,...,...,...,...,...
18251,711.71,1312.70,29.4781,CRC,12df98a7-edfe-478b-a7b7-4f3319630b0d
18252,707.53,1312.40,18.1980,CRC,587eb8b4-80f6-4cfa-8c70-7f10356087c5
18253,721.73,1313.10,56.5445,CRC,7ec39e90-57ec-41a8-98d5-d279948aea12
18254,741.69,1313.80,29.9398,CRC,c51f301a-8dcc-4950-907e-47f70e9654b7


In [36]:
# Load data
# combine the data and metadata file to generate the AnnData object
adata = ad.AnnData (protDF)
adata.obs = spatDF

In [37]:
# Save the results
# Save the results
# Access categorical data
adata.obs['ImageID'] = adata.obs['ImageID'].astype('category')

import hdf5plugin
adata.write('adata.h5ad')

In [38]:
#Function to compute SNR for each protein in each sample
def computeTop20Btm10(ad):
    '''
    Compute the ratio of top 20th percentile to bottom 10th percentile for each protein in each sample
    Input: anndata object
    Output: a dataframe with sampleID, Protein, ratio of top20/btm10
    '''
    top20btm10DF = pd.DataFrame(columns=['ImageID','Protein','top20btm10'])
    # for each sample
    for sID in ad.obs['ImageID'].sort_values().unique():
        subAD = ad[ad.obs.ImageID == sID]
        for x in subAD.var_names:
            aX = subAD[:,x].X.flatten()
            # compute 20 largest values in aX
            top20 = np.sort(aX)[-20:]
            # compute the mean of bottom 10th percentile of aX
            btm10 = np.sort(aX)[:int(len(aX)*0.1)]
            #print(aX.shape, top20.shape, btm10.shape)
            top20btm10 = np.mean(top20)/np.mean(btm10)
            top20btm10DF = pd.concat([top20btm10DF,
                                      pd.DataFrame({'ImageID': [sID], 'Protein': [x], 'top20btm10': [top20btm10]})],
                                     ignore_index=True)
    return top20btm10DF

pd.set_option('display.float_format', lambda x: '%.4f' % x)
result = computeTop20Btm10(ad.read_h5ad('adata.h5ad'))
print(result)

   ImageID   Protein  top20btm10
0      CRC     FoxP3     46.8581
1      CRC      CD68     25.5324
2      CRC      aSMA     25.6301
3      CRC      CD31     71.2149
4      CRC      CD38     27.5051
5      CRC     IDO-1     32.4860
6      CRC      s100     11.4777
7      CRC     CD11c     52.7462
8      CRC     PD-L1     13.1752
9      CRC      Ki67     30.2973
10     CRC       CD8      9.8143
11     CRC      PD-1     10.2021
12     CRC       CD4      9.7124
13     CRC     PanCK     75.3204
14     CRC       CD3     17.5696
15     CRC      CD20    124.9798
16     CRC      CD16     11.7299
17     CRC    HLA-DR     34.9151
18     CRC  Vimentin      7.3815
19     CRC      CD45     22.4459
